In [1]:
import pandas as pd
import seaborn as sns

Описание данных 

- date – дата совершения события

- event - событие

- -- app_install – установка приложения
- -- app_start – открыть приложения
- -- register – зарегистрироваться 
- -- search – перейти на страницу поиска товаров (каталог)
- -- open_item – открыть товар
- -- choose_item – отправить товар в корзину
- -- tap_basket – перейти в корзину
- -- purchase – подтверждение покупки

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

- -- yandex-direct – Яндекс директ
- -- google_ads – реклама в Google
- -- vk_ads – реклама в ВК
- -- instagram_ads – реклама в instagram
- -- facebook_ads – реклама в facebook
- -- referral – акция «приведи друга»
- -- Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

In [2]:
df = pd.read_csv('KC_case_data.csv')

In [3]:
dfc = df.copy()

In [4]:
dfc['date'] = pd.to_datetime(dfc.date)

In [5]:
dfc.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [6]:
# MAU февраля
dfc.loc[dfc.date.dt.month == 2].device_id.nunique()

75032

In [7]:
# установки в январе
dfc .loc[dfc.date.dt.month == 1] \
    .query('event == "app_install"') \
    .device_id.nunique()

80297

In [8]:
install = dfc.query('event == "app_install"')[['date','device_id']] \
             .rename(columns={"date": "date_install"})

In [9]:
install.head()

,date_install,device_id
2515061,2020-01-01,4921563
2515062,2020-01-01,1311583
2515063,2020-01-01,23006325
2515064,2020-01-01,5645652
2515065,2020-01-01,8798567


In [10]:
dfc = dfc.merge(install, how='left', on='device_id')
dfc.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_install
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01


In [11]:
purchase = dfc[dfc.event == 'purchase']
purchase.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_install
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,2020-01-01
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct,2020-01-01
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads,2020-01-01
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads,2020-01-01
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads,2020-01-01


In [12]:
purchase = purchase[(purchase.date - purchase.date_install <= pd.Timedelta('7 days'))]
purchase.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_install
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,2020-01-01
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct,2020-01-01
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads,2020-01-01
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads,2020-01-01
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads,2020-01-01


In [13]:
purchase = purchase.groupby('date_install', as_index=False) \
                   .agg({'device_id': 'nunique'}) \
                   .rename(columns={'device_id': 'purchase'})
purchase.head()

,date_install,purchase
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587


In [14]:
install_group = install.groupby('date_install', as_index=False) \
                        .agg({'device_id': 'nunique'}) \
                        .rename(columns={'device_id': 'installs'})
install_group.head()

,date_install,installs
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671


In [15]:
conversion = install_group.merge(purchase, how='inner', on='date_install')
conversion

,date_install,installs,purchase
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
...,...,...,...
86,2020-03-27,1199,277
87,2020-03-28,1091,209
88,2020-03-29,1117,223
89,2020-03-30,994,204


In [16]:
conversion['CR'] = conversion['purchase'] / conversion['installs']
conversion.head()

,date_install,installs,purchase,CR
0,2020-01-01,3579,1408,0.393406
1,2020-01-02,3144,1186,0.377226
2,2020-01-03,2402,834,0.347211
3,2020-01-04,1831,639,0.348990
4,2020-01-05,1671,587,0.351287


In [17]:
# у какой когорты самый высокий CR

In [18]:
conversion.sort_values(by='CR').tail(1)

,date_install,installs,purchase,CR
0,2020-01-01,3579,1408,0.393406


In [19]:
round(conversion.CR.max() * 100, 1)

39.3

In [20]:
# какой канал принес больше всего новых пользователей
dfc[dfc.event == 'app_install'] \
.groupby('utm_source', as_index=False) \
.device_id.nunique() \
.sort_values('device_id')

,utm_source,device_id
4,referal,9282
1,facebook_ads,13916
3,instagram_ads,20096
5,vk_ads,23189
2,google_ads,26286
6,yandex-direct,29368
0,-,32460


In [21]:
# воронка
dfc.groupby('event', as_index=False).agg({'device_id': 'nunique'}) \
.sort_values('device_id')

,event,device_id
3,purchase,70875
4,register,78310
6,tap_basket,125414
0,app_install,154597
2,choose_item,155691
5,search,184488
1,app_start,190884


In [22]:
# создаем колонку с датой регистрации
register = dfc.query('event == "register"').rename(columns={'date': 'date_reg'})[['date_reg', 'device_id']]
register.head()

,date_reg,device_id
2669658,2020-01-01,294193
2669659,2020-01-01,22917617
2669660,2020-01-01,15248490
2669661,2020-01-01,252062
2669662,2020-01-01,2251583


In [23]:
dfc = dfc.merge(register, how='left', on='device_id')
dfc.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_install,date_reg
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT


In [24]:
dfc_reg = dfc.loc[dfc.date_reg.notna() == True]
dfc_reg.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_install,date_reg
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,2020-01-01,2020-01-24


In [25]:
dfc_reg[dfc_reg.date > dfc_reg.date_reg] \
.groupby('event', as_index=False).agg({'device_id': 'nunique'})

,event,device_id
0,app_start,40991
1,choose_item,37926
2,purchase,24880
3,search,40482
4,tap_basket,34517


In [26]:
dfc.head(10)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_install,date_reg
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,2020-01-01,2020-01-24
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads,2020-01-01,2020-01-23
7,2020-01-01,app_start,NaN,android,8581267,female,Saint-Petersburg,google_ads,2020-01-01,2020-01-13
8,2020-01-01,app_start,NaN,ios,10144701,female,Moscow,-,2020-01-01,2020-01-01
9,2020-01-01,app_start,NaN,ios,10274354,female,Saint-Petersburg,instagram_ads,2020-01-01,NaT


Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [40]:
# Считаем уникальные девайсы с покупками
ch_purch = dfc.query('event == "purchase"') \
              .sort_values('date') \
              .drop_duplicates('device_id', keep='first') \
              .groupby('utm_source', as_index=False) \
              .agg({'device_id': 'nunique'}) \
              .rename(columns={'device_id': 'purchase'})
ch_purch

,utm_source,purchase
0,-,14786
1,facebook_ads,7903
2,google_ads,10167
3,instagram_ads,9820
4,referal,5803
5,vk_ads,11460
6,yandex-direct,10936


In [41]:
# Считаем уникальные девайсы с открытиями по каналам
ch_start = dfc.query('event == "app_start"') \
              .sort_values('date') \
              .drop_duplicates('device_id', keep='first') \
              .groupby('utm_source', as_index=False) \
              .agg({'device_id': 'nunique'}) \
              .rename(columns={'device_id': 'app_start'})
ch_start

,utm_source,app_start
0,-,41456
1,facebook_ads,18844
2,google_ads,31437
3,instagram_ads,24818
4,referal,11983
5,vk_ads,27905
6,yandex-direct,34441


In [42]:
channel = ch_start.merge(ch_purch, how='left', on='utm_source')

In [43]:
channel['cr'] = channel['purchase'] / channel['app_start']
channel.sort_values('cr')

,utm_source,app_start,purchase,cr
6,yandex-direct,34441,10936,0.317529
2,google_ads,31437,10167,0.323409
0,-,41456,14786,0.356667
3,instagram_ads,24818,9820,0.395681
5,vk_ads,27905,11460,0.410679
1,facebook_ads,18844,7903,0.419391
4,referal,11983,5803,0.484269


Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?

In [39]:
dfc.query('event == "purchase"') \
   .sort_values('date') \
   .drop_duplicates('device_id', keep='first') \
   .groupby('utm_source', as_index=False) \
   .agg({'purchase_sum': 'sum'}) \
   .sort_values('purchase_sum')

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

In [55]:
# Бюджеты на маркетинг
dfm = pd.DataFrame({
'utm_source': ['yandex-direct', 'google_ads', 'facebook_ads', 'instagram_ads', 'vk_ads', 'referal'],
'budget': ['10491707', '10534878', '8590498', '8561626', '9553531', '2567800']})
dfm

,utm_source,budget
0,yandex-direct,10491707
1,google_ads,10534878
2,facebook_ads,8590498
3,instagram_ads,8561626
4,vk_ads,9553531
5,referal,2567800


In [64]:
romi = \
dfc.query('event == "purchase"') \
   .groupby('utm_source', as_index=False) \
   .agg({'purchase_sum':'sum','device_id': 'count'}) \
   .merge(dfm, how='left', on='utm_source') \
   .astype({'purchase_sum': 'float', 'budget': 'float'})

In [65]:
romi['romi'] = (romi['purchase_sum'] - romi['budget']) / romi['budget']

In [66]:
romi

,utm_source,purchase_sum,device_id,budget,romi
0,-,21449749.5,30351,NaN,NaN
1,facebook_ads,12249901.0,17219,8590498.0,0.425983
2,google_ads,12868276.0,18479,10534878.0,0.221493
3,instagram_ads,14546969.0,20176,8561626.0,0.699090
4,referal,8837044.5,12839,2567800.0,2.441485
5,vk_ads,16389652.5,22960,9553531.0,0.715560
6,yandex-direct,13915368.0,19359,10491707.0,0.326321
